# Medical Cross-Task Knowledge Transfer - Kaggle Setup

**Project**: Medical NLP with Small Language Models  
**Goal**: Study cross-task knowledge transfer in medical NLP tasks  
**GPU**: T4 (16GB VRAM)  
**Datasets**: ✅ 7 datasets PRE-INCLUDED in repo (no download needed!)

---

## ✅ Datasets Already Included!

All datasets are committed to the repo - just clone and go!

**No more:**
- ❌ Version conflicts
- ❌ Download issues  
- ❌ Library incompatibilities

**Just:**
- ✅ Clone repo
- ✅ Start training!

---

## Setup Checklist

Before running this notebook:
1. ✅ Enable **GPU T4 x2** in Settings → Accelerator
2. ✅ Enable **Internet** in Settings → Internet (for model downloads only)
3. ✅ Set **Persistence** to "Files only" in Settings

---

## 1️⃣ Clone Repository

In [ ]:
# Clone your GitHub repository
!git clone https://github.com/bharathbolla/Crosstalk_Medical_LLM.git
%cd Crosstalk_Medical_LLM

# Verify structure
print("\n📁 Repository structure:")
!ls -la

## 2️⃣ Install Dependencies

In [ ]:
# Install required packages
# Note: We can use latest versions since datasets are pre-included!
!pip install -q transformers datasets evaluate wandb accelerate scikit-learn pyyaml

print("✅ Dependencies installed!")
print("   Using latest versions - no compatibility issues since we're not downloading datasets!")

## 3️⃣ Verify GPU

In [ ]:
import torch

print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print(f"CUDA Version: {torch.version.cuda}")
else:
    print("⚠️ GPU not available! Check Settings → Accelerator → GPU T4 x2")

## 4️⃣ Verify Datasets (Instant!)

All datasets are already in the repo - just verify they loaded correctly.

In [ ]:
from pathlib import Path
from datasets import load_from_disk

# Verify all 7 required datasets exist
data_path = Path("data/raw")

print("📦 Verifying datasets...\n")
print("=" * 60)

datasets_info = {
    "bc2gm": "Gene/protein NER",
    "jnlpba": "Bio-entity NER",
    "ddi": "Drug-drug interaction RE",
    "gad": "Gene-disease association",
    "hoc": "Cancer hallmarks classification",
    "pubmedqa": "Medical QA",
    "biosses": "Sentence similarity"
}

total_samples = 0
verified = 0

for name, description in datasets_info.items():
    print(f"\n📦 {name.upper()} - {description}")
    
    dataset_path = data_path / name
    
    if not dataset_path.exists():
        print(f"   ✗ NOT FOUND at {dataset_path}")
        continue
    
    try:
        # Load the dataset
        dataset = load_from_disk(str(dataset_path))
        
        # Count samples
        train_size = len(dataset["train"]) if "train" in dataset else 0
        total_samples += train_size
        verified += 1
        
        # Show splits
        splits = list(dataset.keys())
        splits_info = ", ".join([f"{s}: {len(dataset[s])}" for s in splits])
        
        print(f"   ✓ Loaded successfully!")
        print(f"   ✓ Splits: {splits_info}")
        
    except Exception as e:
        print(f"   ✗ Error loading: {str(e)[:80]}")

# Summary
print("\n" + "=" * 60)
print(f"✅ Verified: {verified}/7 datasets")
print(f"📊 Total training samples: {total_samples:,}")

if verified == 7:
    print("\n🎉 All datasets ready! You can start training immediately!")
else:
    print(f"\n⚠️  Missing {7 - verified} dataset(s)")
    
print("=" * 60)

In [ ]:
from datasets import load_dataset
from pathlib import Path
import subprocess
import sys

# Create data directory
data_path = Path("data/raw")
data_path.mkdir(parents=True, exist_ok=True)

print("📥 Downloading 7 medical NLP datasets from bigbio collection...\n")
print("=" * 60)

# IMPORTANT: Newer datasets library versions require this
print("⚙️  Setting up environment for bigbio datasets...")
print("   (These datasets use custom loading scripts)")
print()

# Dataset configurations
datasets_config = {
    "bc2gm": {
        "repo": "bigbio/blurb",
        "config": "bc2gm",
        "description": "Gene/protein NER from PubMed abstracts"
    },
    "jnlpba": {
        "repo": "bigbio/blurb",
        "config": "jnlpba",
        "description": "Bio-entity NER (protein, DNA, RNA, cell line, cell type)"
    },
    "ddi": {
        "repo": "bigbio/ddi_corpus",
        "config": "ddi_corpus_source",
        "description": "Drug-drug interaction extraction"
    },
    "gad": {
        "repo": "bigbio/gad",
        "config": "gad_blurb_bigbio_text",
        "description": "Gene-disease association classification"
    },
    "hoc": {
        "repo": "bigbio/hallmarks_of_cancer",
        "config": "hallmarks_of_cancer_source",
        "description": "Cancer hallmarks classification (multi-label)"
    },
    "pubmedqa": {
        "repo": "bigbio/pubmed_qa",
        "config": "pubmed_qa_labeled_fold0_source",
        "description": "Medical question answering"
    },
    "biosses": {
        "repo": "bigbio/biosses",
        "config": "biosses_bigbio_pairs",
        "description": "Biomedical sentence similarity"
    }
}

total_samples = 0
successful = 0
failed = []

for name, config in datasets_config.items():
    print(f"\n📦 {name.upper()}")
    print(f"   {config['description']}")

    try:
        # CRITICAL FIX: Must use trust_remote_code=True for bigbio datasets
        # These datasets have custom loading scripts that are safe but need explicit permission
        dataset = load_dataset(
            config["repo"],
            name=config["config"],
            trust_remote_code=True  # ⚠️ CHANGED from False to True - this is required!
        )

        # Save to disk
        dataset.save_to_disk(str(data_path / name))

        # Show stats
        train_size = len(dataset["train"])
        total_samples += train_size
        successful += 1

        # Show split info
        splits_info = " + ".join([f"{split}: {len(dataset[split])}" for split in dataset.keys()])
        print(f"   ✓ Downloaded! Splits: {splits_info}")

    except Exception as e:
        error_msg = str(e)
        failed.append(name)
        
        # Check if it's the "dataset scripts no longer supported" error
        if "Dataset scripts are no longer supported" in error_msg:
            print(f"   ✗ ERROR: Dataset scripts blocked by datasets library")
            print(f"   💡 FIX: Need to downgrade datasets library")
            print(f"          Run: pip install datasets==2.14.0")
        else:
            print(f"   ✗ ERROR: {error_msg[:100]}")
        continue

# Summary
print("\n" + "=" * 60)
print(f"✅ Successfully downloaded: {successful}/7 datasets")
print(f"📊 Total training samples: {total_samples:,}")

if successful == 7:
    print("\n🎉 All datasets downloaded successfully!")
    print(f"\nDatasets saved in: {data_path.absolute()}")
elif successful == 0:
    print("\n⚠️  FALLBACK NEEDED: All downloads failed!")
    print("\n🔧 FIX: Run this command in a cell BEFORE this one:")
    print("   !pip install -q datasets==2.14.0")
    print("\nThen re-run this cell. The older datasets version supports custom scripts.")
else:
    print(f"\n⚠️  Partial success - Failed datasets: {', '.join(failed)}")
    
print("=" * 60)

## 5️⃣ Test Parsers

In [ ]:
# Test that parsers work
import sys
sys.path.insert(0, "src")

from data import TaskRegistry, BC2GMDataset
from pathlib import Path

# Check registered tasks (should be 7 without ChemProt)
print(f"Registered tasks: {TaskRegistry.list_tasks()}")

# Load one dataset
dataset = BC2GMDataset(
    data_path=Path("data/raw"),
    split="train"
)
print(f"\nLoaded {len(dataset)} BC2GM samples")
print(f"First sample:\n  {dataset[0].input_text[:150]}...")

# Check label schema
schema = dataset.get_label_schema()
print(f"\nLabel schema ({len(schema)} labels): {list(schema.keys())}")

print("\n✅ Everything works! Ready to train!")

## 6️⃣ Smoke Test - Quick Training Test (10 minutes)

Train BERT on 100 samples for 50 steps to verify the pipeline works.

In [ ]:
## 🎉 Success!

If you got here without errors, you're ready for real experiments!

---

## Dataset Summary

All datasets are **pre-included** in the repository:

| Dataset | Task Type | Train Samples | Status |
|---------|-----------|---------------|--------|
| BC2GM | NER | ~15,000 | ✅ Included |
| JNLPBA | NER | ~18,500 | ✅ Included |
| DDI | RE | ~2,900 | ✅ Included |
| GAD | Classification | ~4,200 | ✅ Included |
| HoC | Classification | ~1,500 | ✅ Included |
| PubMedQA | QA | ~450 | ✅ Included |
| BIOSSES | Similarity | 64 | ✅ Included |

**Total**: ~42,000 training samples across 7 diverse medical NLP tasks

**Bonus**: Also includes bc5cdr and chemprot datasets!

---

## Next Steps

### Option 1: Run Test Parsers (1 minute)

Verify all parsers work correctly:

```python
!python test_parsers.py
```

### Option 2: Run Smoke Test (10 minutes)

Quick training test on 100 samples - see Cell 6 above.

### Option 3: Run First Baseline (1 hour)

BERT baseline on BC2GM:

```python
!python scripts/run_baseline.py \
    --model bert-base-uncased \
    --task bc2gm \
    --epochs 3 \
    --batch_size 16
```

### Option 4: Run Full Experiment (4-6 hours)

Single-task training on all tasks:

```python
!python scripts/run_experiment.py strategy=s1_single task=all
```

---

## 📊 Monitor GPU Usage

Run this in a separate cell:

```python
!watch -n 5 nvidia-smi
```

---

## 🔧 Troubleshooting

**"ModuleNotFoundError"**:
- Make sure `sys.path.insert(0, "src")` is in the cell
- Re-run the imports cell

**"CUDA out of memory"**:
- Reduce `per_device_train_batch_size` to 4 or 2
- Add `gradient_accumulation_steps=4` to simulate larger batch

**Session disconnected**:
- Your checkpoints are saved every 200 steps
- Resume training from last checkpoint

**Dataset not found**:
- Make sure you cloned the full repo with `git clone` (not downloaded as ZIP)
- Check `data/raw/` directory exists

---

## 🎯 Time Savings

| Task | Old Method | New Method |
|------|-----------|------------|
| Setup | 8+ hours (with errors) | 2 minutes ✅ |
| Dataset download | Failed | Not needed ✅ |
| Version conflicts | Many | None ✅ |
| Total to first training | Never worked | 15 minutes ✅ |

---

**Happy training!** 🚀

See [DATASETS_INCLUDED.md](DATASETS_INCLUDED.md) for more details.

## 🎉 Success!

If you got here without errors, you're ready for real experiments!

---

## Dataset Summary

| Dataset | Task Type | Samples | Status |
|---------|-----------|---------|--------|
| BC2GM | NER | 12,574 | ✅ Active |
| JNLPBA | NER | 18,607 | ✅ Active |
| ~~ChemProt~~ | ~~RE~~ | ~~1,020~~ | ❌ Disabled |
| DDI | RE | 571 | ✅ Active |
| GAD | Classification | 3,836 | ✅ Active |
| HoC | Classification | 12,119 | ✅ Active |
| PubMedQA | QA | 800 | ✅ Active |
| BIOSSES | Similarity | 64 | ✅ Active |

**Total**: 48,571 samples across 7 diverse tasks

---

## Next Steps

### Option 1: Run Contamination Check (2 hours)

Before training, check if test data leaked into pre-training:

```python
!python scripts/run_contamination_check.py \
    --data_path data/raw \
    --output_dir contamination_results \
    --device cuda
```

### Option 2: Run First Baseline (1 hour)

BERT baseline on BC2GM:

```python
!python scripts/run_baseline.py \
    --model bert-base-uncased \
    --task bc2gm \
    --epochs 3 \
    --batch_size 16
```

### Option 3: Run Full Experiment (4-6 hours)

Single-task training on all tasks:

```python
!python scripts/run_experiment.py strategy=s1_single task=all
```

---

## 📊 Monitor GPU Usage

Run this in a separate cell:

```python
!watch -n 5 nvidia-smi
```

---

## 🔧 Troubleshooting

**"CUDA out of memory"**:
- Reduce `per_device_train_batch_size` to 4 or 2
- Add `gradient_accumulation_steps=4` to simulate larger batch

**"ModuleNotFoundError"**:
- Make sure `sys.path.insert(0, "src")` is in the cell
- Re-run the imports cell

**Session disconnected**:
- Your checkpoints are saved every 200 steps
- Resume training from last checkpoint

---

**Good luck with your experiments!** 🚀